In [3]:
import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np

pn.extension(design='material')

csv_file = (r"..\data\stroke_data.csv")
data = pd.read_csv(csv_file)

data.tail()

,sex,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
40905,1.0,38.0,0,0,0,4,1,120.94,29.7,1,0
40906,0.0,53.0,0,0,1,4,0,77.66,40.8,0,0
40907,1.0,32.0,0,0,1,2,0,231.95,33.2,0,0
40908,1.0,42.0,0,0,1,3,0,216.38,34.5,0,0
40909,1.0,35.0,0,0,0,4,0,95.01,28.0,0,0


In [4]:
def transform_data(variable, iqr_mplr=1.5):
    """ Calculates outliers via IQR """
    q1 = data[variable].quantile(0.25)
    q3 = data[variable].quantile(0.75)
    iqr = q3 - q1
    w1 = q1 - iqr_mplr * iqr
    w3 = q3 + iqr_mplr * iqr
    outliers = (data[variable] > w3) | (data[variable] < w1)
    return data[variable], data[variable][outliers]

def create_plot(variable="avg_glucose_level", iqr_mplr=1.5):
    dots, highlight = transform_data(variable, iqr_mplr=iqr_mplr)
    return dots.hvplot(height=300, width=400, legend=False) * highlight.hvplot.scatter(
        color="orange", padding=0.1, legend=False
    )

In [10]:
create_plot(variable="avg_glucose_level", iqr_mplr=0.1)

:Overlay
   .Curve.Avg_glucose_level   :Curve   [index]   (avg_glucose_level)
   .Scatter.Avg_glucose_level :Scatter   [index]   (avg_glucose_level)

In [13]:
variable_widget = pn.widgets.Select(name='variable', value='avg_glucose_level', options=list(data.columns))
iqr_mplr_widget = pn.widgets.FloatSlider(name='iqr_mplr', value=1.5, start=0.01, end=2.0)

In [14]:
bound_plot = pn.bind(create_plot, variable=variable_widget, iqr_mplr=iqr_mplr_widget)

In [15]:
first_app = pn.Column(variable_widget, iqr_mplr_widget, bound_plot)

first_app.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'63268051-2fcc-47bc-b2ba-67d09757d0ef': {'version…